In [8]:
import os
import re
import time

try:
    import PyPDF2
except ModuleNotFoundError:
    !pip install PyPDF2
    
try:
    import openai
except ModuleNotFoundError:
    !pip install openai

In [9]:
#Write paper link   
url="https://arxiv.org/pdf/1912.04958.pdf"

#Choose the preferred language
lang="English"

#Give a name to your summary (optional)
article_name="paper.pdf"

In [10]:
!curl -s -o $article_name $url
# Set the string that will contain the summary     
pdf_summary_text = ""
# Read the PDF file using PyPDF2
pdf_file = open(article_name, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [11]:
def ask(demand):
    # Call the OpenAI API to generate a response to the 'demand' parameter using a GPT-3.5 model called "gpt-3.5-turbo".
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        # Set the context by sending a system message to the model informing the user that they are a helpful research assistant.
        {"role": "system", "content": "You are a helpful research assistant."},
        # Send the 'demand' parameter as a user message to the model.
        {"role": "user", "content": demand},
        ],
    )
    # Return the generated response as a string. The content of the first message choice returned by the API response.
    return response["choices"][0]["message"]["content"]

In [ ]:
page_sums=[]

# Loop through all the pages in the PDF file
for page_num in range(len(pdf_reader.pages)):
    # Extract the text from the page
    page_text = pdf_reader.pages[page_num].extract_text().lower()

    max_retries = 10
    retries = 0
    while retries < max_retries:
        try:
            demand = f"Summarize this in {lang} : {page_text}"
            page_summary = ask(demand)
            page_sums.append(page_summary)
            print("Summarizing page " + str(page_num))
            break  # If the API call is successful, break the loop
        except openai.errors.RateLimitError as e:
            print(f"Caught RateLimitError: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Waiting for 10 seconds before retrying. Retry attempt {retries}/{max_retries}.")
                time.sleep(10)
            else:
                print(f"Reached maximum retries ({max_retries}). Exiting.")
                break
    else:
        # If the loop has finished and no successful API call was made, skip this iteration
        continue

print("Done!")

Summarizing page 0
Summarizing page 1
Summarizing page 2
Summarizing page 3
Summarizing page 4
Summarizing page 5
Summarizing page 6


In [ ]:
pdf_summary_text='\n'.join(page_sums)
page_to_page_summary = article_name.replace(os.path.splitext(article_name)[1], ".ptp_summary.txt")

# Create the page summaries
with open(page_to_page_summary, "w+", encoding="utf-8") as file:
    file.write(pdf_summary_text)
print(pdf_summary_text)
file.close()

In [ ]:
#Give minimum words for your summary
minimum_words=300

ultimate_summary = ""
page_limit = 5
division=int(len(page_sums) / page_limit)

if len(page_sums) > page_limit:
    # divide the list into smaller parts
    smaller_lists = [page_sums[i:i + page_limit] for i in range(0, len(page_sums), page_limit)]

    # concatenate the items in each smaller list
    concat_list = [''.join(smaller_list) for smaller_list in smaller_lists]

    for i in concat_list:
        first_summary = ' '.join(pdf_summary_text.split(' ')[:500])
        
        # Generate a summary for each smaller list and concatenate them
        demand = f"Create summary of this text in {lang} consisting maximum {minimum_words} words. {first_summary}"
        ultimate_summary_part = ask(demand)
        ultimate_summary += ultimate_summary_part + "\n"
    
    # Generate a final summary from the concatenated summaries    
    demand = f"Create summary of this text in {lang} from these page summaries. Calculate tokens left and use maximum tokens you can.   {ultimate_summary}"
    ultimate_summary_part = ask(demand)
    ultimate_summary_file = article_name.replace(os.path.splitext(article_name)[1], ".summary.txt")

else:

    first_summary = ' '.join(pdf_summary_text.split(' ')[:500])
    demand = f"Create a summary of this text in {lang}. The text must have minimum {minimum_words} words. Calculate tokens left and use maximum tokens you can. {first_summary}"
    ultimate_summary = ask(demand)
    ultimate_summary_file = article_name.replace(os.path.splitext(article_name)[1], ".summary.txt")
    
with open(ultimate_summary_file, "w+", encoding="utf-8") as file2:
    file2.write(ultimate_summary)

print(ultimate_summary)
file2.close()
pdf_file.close()